USECASE-2 DYNAMIC DELIVERY SCHEDULING - AGENT-2

Pre-requisites:

#This notebook requires permissions to:
- create and delete Amazon IAM roles
- access Amazon Bedrock

#The following access are required for Sagemaker to carry on the usecase:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonBedrockFullAccess
- AmazonDynamoDBFullAccess

EDA and extraction of required columns

In [61]:
import pandas as pd

# Load the dataset
csv_file = r"Supply chain dataset - Supply chain dataset (2).csv"
data = pd.read_csv(csv_file)

def get_delivery_scheduling_summary():
    # Filter the data for entries where ProductName is 'Laptop'
    laptop_data = data[data['PurchaseHistory'] == 'Smartwatch']

    # Specify the relevant columns
    relevant_columns = [
        'OrderID', 'OrderVolume', 'DeliveryLocation', 'DeliveryTime(days)', 
        'StartLocation', 'TrafficCondition', 'SupplierName' ,
        'PurchaseHistory', 'ReorderQuantity', 'PromotionalEvents', 'AnomalyDetected', 'AnomalyType'
    ]
    
    delivery_info = laptop_data[relevant_columns]

    # Combine the information into a summary string
    summaries = []
    for _, row in delivery_info.iterrows():
        summary = (
            f"OrderID {row['OrderID']} with volume {row['OrderVolume']} is to be delivered to {row['DeliveryLocation']} "
            f"from {row['StartLocation']}. Expected delivery time is {row['DeliveryTime(days)']} days. "
            f"SupplierName {row['SupplierName']} (Status: {row['SupplierName']}) "
           
            f"Reorder quantity is {row['ReorderQuantity']}. Promotional event: {row['PromotionalEvents']}. "
            f"Anomaly detected: {row['AnomalyDetected']} (Type: {row['AnomalyType']})."
        )
        summaries.append(summary)
    
    result = " ".join(summaries)
    return result

summary = get_delivery_scheduling_summary()
print(summary)


OrderID 38376 with volume 67 is to be delivered to Pune from Warehouse_Main. Expected delivery time is 3 days. SupplierName Supplier_A (Status: Supplier_A) Reorder quantity is 76. Promotional event: None. Anomaly detected: True (Type: Delay). OrderID 13698 with volume 28 is to be delivered to Pune from Warehouse_Main. Expected delivery time is 6 days. SupplierName Supplier_B (Status: Supplier_B) Reorder quantity is 83. Promotional event: Event_2. Anomaly detected: True (Type: None). OrderID 49063 with volume 79 is to be delivered to Mumbai from Warehouse_Main. Expected delivery time is 8 days. SupplierName Supplier_F (Status: Supplier_F) Reorder quantity is 34. Promotional event: Event_2. Anomaly detected: True (Type: None). OrderID 25094 with volume 20 is to be delivered to Kolkata from Warehouse_Main. Expected delivery time is 3 days. SupplierName Supplier_D (Status: Supplier_D) Reorder quantity is 55. Promotional event: None. Anomaly detected: True (Type: None). OrderID 42946 with v

CREATING AGENT

In [25]:
!pip install boto3
!pip install opensearch-py
!pip install botocore
!pip install awscli
!pip install retrying

In [56]:
import boto3
import json
import time
import zipfile
import logging
import pprint
from io import BytesIO

iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)





def invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False, session_state=None):
    end_session: bool = False
    if not session_state:
        session_state = {}

    # invoke the agent API
    agent_response = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=enable_trace,
        endSession=end_session,
        sessionState=session_state
    )

    if enable_trace:
        logger.info(pprint.pprint(agent_response))

    event_stream = agent_response['completion']
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)


def create_agent_role(agent_name, agent_foundation_model, kb_id=None):
    agent_bedrock_allow_policy_name = f"{agent_name}-ba"
    agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
    # Create IAM policies for agent
    statements = [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
    # add Knowledge Base retrieve and retrieve and generate permissions if agent has KB attached to it
    if kb_id:
        statements.append(
            {
                "Sid": "QueryKB",
                "Effect": "Allow",
                "Action": [
                    "bedrock:Retrieve",
                    "bedrock:RetrieveAndGenerate"
                ],
                "Resource": [
                    f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"
                ]
            }
        )

    bedrock_agent_bedrock_allow_policy_statement = {
        "Version": "2012-10-17",
        "Statement": statements
    }

    bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)
    try:
        agent_bedrock_policy = iam_client.create_policy(
            PolicyName=agent_bedrock_allow_policy_name,
            PolicyDocument=bedrock_policy_json
        )
    except iam_client.exceptions.EntityAlreadyExistsException:
        agent_bedrock_policy = iam_client.get_policy(
            PolicyArn=f"arn:aws:iam::{account_id}:policy/{agent_bedrock_allow_policy_name}"
        )
                    
    # Create IAM Role for the agent and attach IAM policies
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)
    try:
        agent_role = iam_client.create_role(
            RoleName=agent_role_name,
            AssumeRolePolicyDocument=assume_role_policy_document_json
        )

        # Pause to make sure role is created
        time.sleep(10)
    except iam_client.exceptions.EntityAlreadyExistsException:
        agent_role = iam_client.get_role(
            RoleName=agent_role_name,
        )

    iam_client.attach_role_policy(
        RoleName=agent_role_name,
        PolicyArn=agent_bedrock_policy['Policy']['Arn']
    )
    return agent_role


def delete_agent_roles_and_policies(agent_name, kb_policy_name):
    agent_bedrock_allow_policy_name = f"{agent_name}-ba"
    agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
    dynamodb_access_policy_name = f'{agent_name}-dynamodb-policy'
    lambda_function_role = f'{agent_name}-lambda-role'

    for policy in [agent_bedrock_allow_policy_name, kb_policy_name]:
        try:
            iam_client.detach_role_policy(
                RoleName=agent_role_name,
                PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
            )
        except Exception as e:
            print(f"Could not detach {policy} from {agent_role_name}")
            print(e)

    for policy in [dynamodb_access_policy_name]:
        try:
            iam_client.detach_role_policy(
                RoleName=lambda_function_role,
                PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
            )
        except Exception as e:
            print(f"Could not detach {policy} from {lambda_function_role}")
            print(e)

    try:
        iam_client.detach_role_policy(
            RoleName=lambda_function_role,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )
    except Exception as e:
        print(f"Could not detach AWSLambdaBasicExecutionRole from {lambda_function_role}")
        print(e)

    for role_name in [agent_role_name, lambda_function_role]:
        try:
            iam_client.delete_role(
                RoleName=role_name
            )
        except Exception as e:
            print(f"Could not delete role {role_name}")
            print(e)

    for policy in [agent_bedrock_allow_policy_name, kb_policy_name, dynamodb_access_policy_name]:
        try:
            iam_client.delete_policy(
                PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
            )
        except Exception as e:
            print(f"Could not delete policy {policy}")
            print(e)



In [57]:
import time
import boto3
import logging
import ipywidgets as widgets
import uuid




In [58]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-east-1', '199328835594')

In [8]:
suffix = f"{region}-{account_id}"
agent_name = 'delivery-opt-sch-awshack'
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

agent_description = "Agent in charge of going through complete data and refer the best routes and method to delivery for Laptops"
agent_instruction = """
You are a supply chain management agent. you go through the complete dataset and give details on route optimization for various products and suppliers.
"""

agent_action_group_description = """
creating delivery optimisation"""    

agent_action_group_name = "optimiseddelivery"

In [9]:
agent_foundation_model = 'anthropic.claude-3-sonnet-20240229-v1:0'
agent_foundation_model

'anthropic.claude-3-sonnet-20240229-v1:0'

In [10]:
agent_role = create_agent_role(agent_name, agent_foundation_model)

In [11]:
agent_role

{'Role': {'Path': '/',
  'RoleName': 'AmazonBedrockExecutionRoleForAgents_delivery-opt-sch-awshack',
  'RoleId': 'AROAS42HMVAFEJAMOINMZ',
  'Arn': 'arn:aws:iam::199328835594:role/AmazonBedrockExecutionRoleForAgents_delivery-opt-sch-awshack',
  'CreateDate': datetime.datetime(2024, 8, 26, 7, 26, 53, tzinfo=tzlocal()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'bedrock.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]}},
 'ResponseMetadata': {'RequestId': 'd05e7ee5-d643-4627-8458-71f3d8832601',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 26 Aug 2024 07:26:53 GMT',
   'x-amzn-requestid': 'd05e7ee5-d643-4627-8458-71f3d8832601',
   'content-type': 'text/xml',
   'content-length': '875'},
  'RetryAttempts': 0}}

In [12]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

{'ResponseMetadata': {'RequestId': '615bdfa3-3f75-4bd7-8557-7c53a6c62352',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 26 Aug 2024 07:27:20 GMT',
   'content-type': 'application/json',
   'content-length': '761',
   'connection': 'keep-alive',
   'x-amzn-requestid': '615bdfa3-3f75-4bd7-8557-7c53a6c62352',
   'x-amz-apigw-id': 'dGwtxGBMIAMEtXQ=',
   'x-amzn-trace-id': 'Root=1-66cc2e57-11f462b2444683155c5e3dba'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:199328835594:agent/UZ6TI8TTWF',
  'agentId': 'UZ6TI8TTWF',
  'agentName': 'delivery-opt-sch-awshack',
  'agentResourceRoleArn': 'arn:aws:iam::199328835594:role/AmazonBedrockExecutionRoleForAgents_delivery-opt-sch-awshack',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 8, 26, 7, 27, 19, 996982, tzinfo=tzlocal()),
  'description': 'Agent in charge of going through complete data and refer the best routes and method to delivery for Laptops',
  'foundationModel': 'anthropic.cla

In [34]:
agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

TypeError: string indices must be integers

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

INVOKING AND TESTING

In [62]:
alias_id = 'TSTALIASID'
agent_id = "UZ6TI8TTWF"

In [63]:
%%time
session_id:str = str(uuid.uuid1())
query = "only give me the 2 top best routes and best supplierID(based on their reliability) AND best delivery schedule to deliver a smartwatch to Delhi" + summary 

response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

The top 2 best routes for delivering the smartwatch to Delhi would be:
1. Warehouse_Main -> Lucknow (via Supplier_A)
2. Warehouse_Main -> Jaipur (via Supplier_A)

The best supplierID based on reliability is Supplier_A.

For OrderID 38376 with volume 67 to be delivered to Pune from Warehouse_Main with an expected delivery time of 3 days, the best delivery schedule would be to use Supplier_A, but account for potential delays due to the anomaly detected of type Delay.
CPU times: user 24.5 ms, sys: 525 μs, total: 25 ms
Wall time: 14.3 s
